In [1]:
!pip install git+https://github.com/jotform/jotform-api-python.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jotform/jotform-api-python.git to /tmp/pip-req-build-o59j3eks
  Running command git clone --filter=blob:none --quiet https://github.com/jotform/jotform-api-python.git /tmp/pip-req-build-o59j3eks
  Resolved https://github.com/jotform/jotform-api-python.git to commit 050e92db72c315496e335be4959307e6a7043ed2
  Preparing metadata (setup.py) ... done
  Created wheel for jotform: filename=jotform-1.2-py3-none-any.whl size=12361 sha256=99d6a2c726e5da40cd5e883c69ee94c2c94618e5f41fc2f047af72d3365e133c
  Stored in directory: /tmp/pip-ephem-wheel-cache-h_eo2quq/wheels/f4/90/54/22d595a2e6b1a78968b00a27607fdab29a6b2d271e00185d36
Successfully built jotform


In [2]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import requests
from jotform import JotformAPIClient

def main():
    # Change this API KEY with your Jotform Account API KEY
    API_Key = 'a8dee57ad07b6de1bd49f438e709d79e'	
    jotformAPIClient = JotformAPIClient(API_Key)

    forms = jotformAPIClient.get_forms(None, 1, None, None)
    latestForm = forms[0]
    latestFormID = latestForm["id"]

    submissions = jotformAPIClient.get_form_submissions(latestFormID)

    image_links = []

    for submission in submissions:
        answers = submission['answers']
        for key, value in answers.items():
            if 'answer' in value and 'text' in value:
                answer_text = value['text']
                answer_link = value['answer']
                found_text = False

                for entry in image_links:
                    if entry[0] == answer_text:
                        entry[1].append(answer_link)
                        found_text = True
                        break

                if not found_text:
                    image_links.append([answer_text, [answer_link]])

    output_folder = 'image_data'
    os.makedirs(output_folder, exist_ok=True)

    for entry in image_links:
        text_value = entry[0]
        links = entry[1]
        text_folder = os.path.join(output_folder, text_value)
        os.makedirs(text_folder, exist_ok=True)

        for link in links:
            file_info = link.split('/')
            username = file_info[4]
            form_id = file_info[5]
            submission_id = file_info[6]
            file_name = file_info[7]

            download_url = f"https://www.jotform.com/uploads/{username}/{form_id}/{submission_id}/{file_name}"

            image_path = os.path.join(text_folder, file_name)

            response = requests.get(download_url)
            if response.status_code == 200:
                with open(image_path, 'wb') as file:
                    file.write(response.content)
                    print(f"Downloaded {file_name} for {text_value}")
            else:
                print(f"Failed to download {file_name} for {text_value}")

    print("All files downloaded successfully!")

if __name__ == "__main__":
    main()


In [9]:
import shutil

def zip_folder(folder_path, zip_path):
    shutil.make_archive(zip_path, 'zip', folder_path)

# ZIP Image data to be downloaded
folder_path = 'image_data'
zip_path = 'image_data'
zip_folder(folder_path, zip_path)